In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz

import difflib 
from fuzzywuzzy import process


/opt/homebrew/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [4]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

In [62]:
#TODO: see what's up with wcvp
def find_lower_case_wcvp(vol):
    author_set = set()
    for s in vol['authors']:
        if isinstance(s, str):
            for w in list(s.split()):
                author_set.add(w)

    author_set_lower_case = set()
    for s in author_set:
        pruned_s = ""
        for c in s:
            if not c.isalnum() and (c != "." or c != "(" or c != ")"):
                pruned_s += c
                author_set_lower_case.add(s)
        # if pruned_s.islower():
        #     author_set_lower_case.add(s)
    return author_set_lower_case

find_lower_case_wcvp(wcvp_df)

{'S.C.Brooks',
 '(C.T.Lu',
 'Schkr.',
 'Ferriol,',
 'E.Hummel',
 '(B.Øllg.)',
 'R.C.Römer',
 '(U.Müll.-Doblies)',
 '(Maekawa)',
 'Schmalh.',
 'M.R.Schomb.',
 'C.H.Zhao',
 '(Gaterau)',
 'X.Y.Wang',
 'B.R.Arrill.',
 'R.Zhang',
 'J.G.M.Pers.',
 'C.W.Riggins',
 'G.Dumont',
 'M.Y.Fang)',
 '(Stapf',
 'K.Ueda',
 'Z.J.Li',
 'Uhlich,',
 'Cubas,',
 '(Z.L.Yang)',
 '(Sundell)',
 '(T.F.Patt.)',
 'Daly,',
 '(Garay',
 'O.Berg',
 '(K.S.Hao)',
 'Panz.',
 'R.C.K.Chung',
 'A.K.Neill',
 'Vigo,',
 'Gust.',
 '(Ehlers',
 'Friesn.',
 '(Henriq.)',
 '(Frankis)',
 'Cogollo,',
 '(L.Touss.)',
 'F.Wang',
 'G.Dietr.',
 'Kaczm.',
 'J.E.Benedict',
 'Rib.-Silva',
 'H.Y.Ye',
 'L.P.Queiroz,',
 '(B.Boivin)',
 'Urdampill.)',
 'Q.H.Liu',
 'Gajurel,',
 'Yild.',
 '(Kodama)',
 'Heek,',
 'C.H.Teo',
 '(Chrtek,',
 'M.E.Jones',
 'Schlindw.',
 'Y.G.Shen',
 '(Gugler)',
 '(R.R.Stewart)',
 '(Barceló)',
 'Sibth.',
 'Mor.,',
 'Podlech,',
 'Bedd.)',
 '(J.R.Lisboa)',
 'Boll)',
 'B.K.Nayar,',
 '(R.Tracey)',
 '(Sommier)',
 'Merr.)',
 'J.R.H

In [8]:
def find_lower_case(vol):
    author_set = set()
    for s in vol['mouterde_author']:
        if isinstance(s, str):
            for w in list(s.split()):
                author_set.add(w)

    author_set_lower_case = set()
    for s in author_set:
        pruned_s = ""
        for c in s:
            if c.isalnum():
                pruned_s += c
        if pruned_s.islower():
            author_set_lower_case.add(s)
    return author_set_lower_case

find_lower_case(vol3_df): 

{'adoxifolium',
 'agg.',
 'anagallis-aquatica',
 'aperta',
 'asperrimus',
 'assurense',
 'auct.',
 'benedictus',
 'cassia',
 'colocynthis',
 'cornucopiae',
 'coronarium',
 'discoidea',
 'emend.',
 'et',
 'ex',
 'f.',
 'fa',
 'fii.',
 'fil.',
 'fil.)',
 'hispidula',
 'i-à*',
 'in',
 'j.',
 'juncea',
 'kotschyi',
 'lasianthum',
 'leiocarpa',
 'lysimachioides',
 'mult.',
 'nec',
 'non',
 'nty)',
 'orientalis',
 'pentagonia',
 'phelipaea',
 'ptychophyllum',
 's.l.',
 'sinuatum',
 'tenuis',
 'tripolitanum',
 'urvilleana',
 'yar.'}

In [11]:
find_lower_case(vol2_df)

{'(li.)',
 'er',
 'et',
 'ex',
 'f.',
 'fil.',
 'fil.)',
 'lentiscus',
 'maximus',
 'nova',
 'ri',
 'ssp.',
 'tris'}

In [25]:
vol2_df['mouterde_author'].dropna

<bound method Series.dropna of 0                   NaN
1                  Del.
2           (L.) Willd.
3                   NaN
4       Bornm. et Schw.
             ...       
1458              Vill.
1459      Boiss. et Bo.
1460           M. Bieb.
1461              Poir.
1462             Boiss.
Name: mouterde_author, Length: 1463, dtype: object>

In [33]:
vol2_df[vol2_df['mouterde_author'].str.contains('tris') == True]

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
1042,718,Lavatera,trimes,NaN,tris L.,species


In [12]:
find_lower_case(vol1_df)

{'et', 'f.)', 'fil.', 'otica', 'syriaci', 't.', 'turn', 'ﬁoiss.'}

In [ ]:
"""find_lower_case(vol3_df): 

{'adoxifolium',
 'agg.',
 'anagallis-aquatica',
 'aperta',
 'asperrimus',
 'assurense',
 'auct.',
 'benedictus',
 'cassia',
 'colocynthis',
 'cornucopiae',
 'coronarium',
 'discoidea',
 'emend.',
 'et',
 'ex',
 'f.',
 'fa',
 'fii.',
 'fil.',
 'fil.)',
 'hispidula',
 'i-à*',
 'in',
 'j.',
 'juncea',
 'kotschyi',
 'lasianthum',
 'leiocarpa',
 'lysimachioides',
 'mult.',
 'nec',
 'non',
 'nty)',
 'orientalis',
 'pentagonia',
 'phelipaea',
 'ptychophyllum',
 's.l.',
 'sinuatum',
 'tenuis',
 'tripolitanum',
 'urvilleana',
 'yar.'}"""

In [53]:
vol3_df[vol3_df['mouterde_author'].str.contains("mult.") == True]

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
1467,579,Vaillantia,NaN,NaN,auct. mult. non L.,genus


In [54]:
vol3_df[vol3_df['mouterde_author'].str.contains("nec") == True]

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
1522,580,Verbascum,glomeratum,NaN,"Boiss. Moretti, nec Griseb., nec Velen.",species


In [55]:
vol3_df[vol3_df['mouterde_author'].str.contains("non") == True]

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
807,568,Linaria,parviflora,NaN,Desf. non Jacq.,species
1256,576,Scutellaria,sibthorpii,NaN,auct. non (Benth.) Hal.,species
1467,579,Vaillantia,NaN,NaN,auct. mult. non L.,genus


In [56]:
vol3_df[vol3_df['mouterde_author'].str.contains("s.l.") == True]

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
49,556,Anacyclus,radiatus,NaN,Loisel.,species
251,559,Carduus,nutans,NaN,L. s.l.,species
256,559,Carlina,curetum,orientalis,Meusel et Kâstner,subsp.
258,559,Carlina,hispanica,galilaea,Meusel et Kàstner,subsp.
260,559,Carlina,involucrata,libanotica,(Boiss.) Meusel et,subsp.
261,559,Carlina,kurdica,NaN,Meusel et Kâstner,species


In [ ]:
vol3_df[vol3_df['mouterde_author'].str.contains("yar.") == True]

In [59]:
vol3_df[vol3_df['mouterde_epithet'] == 'santohna']

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank
